In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import os

In [2]:
mnist = input_data.read_data_sets('../../../dataset/MNIST', one_hot=True)
Z_dim = 128
batch_size = 64

# define inputs
with tf.variable_scope("inputs"):
    X = tf.placeholder(tf.float32, shape= [None, mnist.train.images.shape[1]])
    Z = tf.placeholder(tf.float32, shape= [None, Z_dim])
    Y = tf.placeholder(tf.float32, shape= [None, mnist.train.labels.shape[1]])

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../../dataset/MNIST\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../../dataset/MNIST\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../../../dataset/MNIST\t10k-images-idx3-ubyte.gz
Extracting ../../../dataset/MNIST\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
def generator(z, y):
    with tf.variable_scope("generator"):
        inputs = tf.concat(values=[z, y],axis=1)
        G_h1 = tf.layers.dense(inputs,
                               128,
                               activation=tf.nn.relu,
                               kernel_initializer=tf.glorot_normal_initializer())
        G_prob = tf.layers.dense(G_h1, 784, activation = tf.nn.sigmoid, kernel_initializer=tf.glorot_normal_initializer())
    
        return G_prob

def discriminator(x, y, reuse):
    with tf.variable_scope("discriminator"):       
        inputs = tf.concat(values=[x, y],axis=1)
        D_h1 = tf.layers.dense(inputs,
                               128,
                               reuse = reuse,
                               activation=tf.nn.relu,
                               kernel_initializer=tf.glorot_normal_initializer(), 
                               name = "L1")
        D_logit = tf.layers.dense(D_h1, 1, reuse = reuse, kernel_initializer=tf.glorot_normal_initializer(), name = "L2")
        D_prob = tf.nn.sigmoid(D_logit)
        return D_prob, D_logit

def loss(D_logit_real, D_logit_fake):
    # D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
    # G_loss = -tf.reduce_mean(tf.log(D_fake))
    
    # Alternative losses:
    # -------------------
    D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
    D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
    D_loss = D_loss_real + D_loss_fake
    G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))
    
    return G_loss, D_loss

def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

def plot(samples,inputs):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')
    fig.suptitle(inputs)
    return fig

In [4]:
G_sample = generator(Z, Y)
D_real, D_logit_real = discriminator(X, Y, reuse=None)
D_fake, D_logit_fake = discriminator(G_sample, Y, reuse=True)

In [5]:
t_vars = tf.trainable_variables()
g_vars = [var for var in t_vars if var.name.startswith("generator")]
d_vars = [var for var in t_vars if var.name.startswith("discriminator")]

In [6]:
G_loss, D_loss = loss(D_logit_real, D_logit_fake)
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=d_vars)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=g_vars)

In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('out/'):
    os.makedirs('out/')

for it in range(50000):
    X_mb, Y_mb = mnist.train.next_batch(batch_size) 
    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb,
                                                             Z: sample_Z(batch_size, Z_dim),
                                                             Y: Y_mb})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(batch_size, Z_dim),
                                                             Y: Y_mb})

    if it % 1000 == 0:
        print('Iter: {}, D loss: {:.4}, G_loss: {:.4}'.format(it,D_loss_curr,G_loss_curr))
        
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim),
                                               Y: Y_mb[0:16,]})

        fig = plot(samples, str(np.argmax(Y_mb[0:16,], axis=1)))
        plt.savefig('out/ex05_CGAN/{}.png'.format(str(it).zfill(5)), bbox_inches='tight')
        plt.close(fig)

Iter: 0, D loss: 1.497, G_loss: 2.314
Iter: 1000, D loss: 0.02318, G_loss: 4.699
Iter: 2000, D loss: 0.03096, G_loss: 5.852
Iter: 3000, D loss: 0.09164, G_loss: 4.775
Iter: 4000, D loss: 0.09504, G_loss: 4.526
Iter: 5000, D loss: 0.2952, G_loss: 4.593
Iter: 6000, D loss: 0.4017, G_loss: 4.241
Iter: 7000, D loss: 0.4038, G_loss: 3.596
Iter: 8000, D loss: 0.4699, G_loss: 3.919
Iter: 9000, D loss: 0.6753, G_loss: 3.367
Iter: 10000, D loss: 0.5942, G_loss: 2.644
Iter: 11000, D loss: 0.5893, G_loss: 2.603
Iter: 12000, D loss: 0.6326, G_loss: 2.898
Iter: 13000, D loss: 0.7065, G_loss: 1.916
Iter: 14000, D loss: 0.8715, G_loss: 2.159
Iter: 15000, D loss: 0.7957, G_loss: 2.549
Iter: 16000, D loss: 0.7567, G_loss: 1.902
Iter: 17000, D loss: 0.8535, G_loss: 2.199
Iter: 18000, D loss: 0.6765, G_loss: 1.771
Iter: 19000, D loss: 0.8354, G_loss: 2.283
Iter: 20000, D loss: 0.7383, G_loss: 1.829
Iter: 21000, D loss: 0.8343, G_loss: 2.26
Iter: 22000, D loss: 0.8736, G_loss: 1.392
Iter: 23000, D loss: 0